In [32]:
import json
import numpy as np
from collections import Counter

In [2]:
samples = json.load(open('../jamesbot/data/processed/user_simulator/samples.json', 'r'))

In [129]:
action_messages = {}
dialog_templates = []

for sample in samples:
    dialog_constraints = {}
    dialog_requests = set()
    for turn in sample:
        if turn['author'] != 'user':
            continue
        text = turn['text']

        for (slot, value) in turn['slot_values'].items():
            if slot not in {'intent'}:
                text = text.replace(str(value), '<SLOT.'+slot+'>')
                
                if slot not in dialog_constraints:
                    dialog_constraints[slot] = []
                
                if len(dialog_constraints[slot]) > 0:
                    if not dialog_constraints[slot][-1] == value:
                        dialog_constraints[slot].append(value)
                else:
                    dialog_constraints[slot].append(value)
                    
        actions, action_slots = set(), set()
        for (action, slots) in turn['action_slots'].items():
            actions.add(action)
            action_slots = action_slots.union(set(slots))
            if action == 'request':
                dialog_requests = dialog_requests.union(set(slots))
        action_key = '&'.join(sorted(actions))+'.'+'&'.join(sorted(action_slots))
        
        if action_key not in action_messages:
            action_messages[action_key] = Counter()
        
        action_messages[action_key][text] += 1
    
    dialog_templates.append({
        'constraints': dialog_constraints,
        'requests': list(dialog_requests)
    })

templates = {}

for (action, messages) in action_messages.items():
    templates[str(action)] = list(map(lambda m: m[0], messages.most_common(10)))
    
with open('../jamesbot/data/processed/user_simulator/message_templates.json', 'w') as f:
    json.dump(templates, f)

with open('../jamesbot/data/processed/user_simulator/dialog_templates.json', 'w') as f:
    json.dump(dialog_templates, f)

In [130]:
len(dialog_templates)

1369